In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imread
import os
import seaborn as sns
import tensorflow
import keras
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.models import Model

In [67]:
from tensorflow.keras.applications.resnet_v2 import ResNet101V2 as ResNet
from tensorflow.keras.applications.densenet import DenseNet121 as DenseNet
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inceptionNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as MobileNet

In [68]:
def create_Model():
    input_ = Input(shape = (640,640,3))
    
    Res = ResNet(include_top=False, weights='imagenet', input_tensor=input_, pooling='average',classifier_activation='softmax')
    Inception = inceptionNet(include_top = False, weights = 'imagenet', input_tensor = input_, pooling = 'average', classifier_activation = 'softmax')
    
    Inception_mid_In = Inception.get_layer('mixed7').output
    Inception_mid_pad = ZeroPadding2D(padding = (1,1))(Inception_mid_In)
    
    Res_mid_In = Res.get_layer('conv4_block7_out').output
    
    mid_concatenation = tensorflow.keras.layers.Concatenate(axis = -1)([Res_mid_In, Inception_mid_pad])
    mid_concatenation_conv2D1 = Conv2D(1024, (1,1), activation='relu', padding='same')(mid_concatenation)
    mid_concatenation_padding = ZeroPadding2D(padding = (100,100))(mid_concatenation_conv2D1)
    mid_concatenation_conv2D2 = Conv2D(3, (1,1), activation='relu', padding='same')(mid_concatenation_padding)
    
    
    MNet = MobileNet(include_top = False, weights = None, input_tensor = mid_concatenation_conv2D2, pooling = 'average', classifier_activation = 'softmax')
    MNet_Out_flatten = Flatten()(MNet.output)
    MNet_Out_Dense = Dense(3, activation = 'sigmoid')(MNet_Out_flatten)
    
    
    
    Res_Out_flatten = Flatten()(Res.output)
    Res_Out_Dense = Dense(3, activation = 'sigmoid')(Res_Out_flatten)
    
    Inception_Out_flatten = Flatten()(Inception.output)
    Inception_Out_Dense = Dense(3, activation = 'sigmoid')(Inception_Out_flatten)
    
    concatenation_out = tensorflow.keras.layers.Concatenate(axis = -1)([Res_Out_Dense, Inception_Out_Dense, MNet_Out_Dense])
    
    output = Dense(3, activation='sigmoid', name='output')(concatenation_out)
    model = Model(input_, output)
    
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model


In [69]:
model = create_Model()
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_31 (InputLayer)          [(None, 640, 640, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 646, 646, 3)  0           ['input_31[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 320, 320, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

 conv2_block2_out (Add)         (None, 160, 160, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 160, 160, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 160, 160, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 160, 160, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 conv3_block2_1_relu (Activatio  (None, 80, 80, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 82, 82, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 80, 80, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 80, 80, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block1_preact_bn (BatchN  (None, 40, 40, 512)  2048       ['conv3_block4_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 40, 40, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 40, 40, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_blo

 conv4_block2_2_bn (BatchNormal  (None, 40, 40, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 max_pooling2d_199 (MaxPooling2  (None, 158, 158, 64  0          ['activation_2634[0][0]']        
 D)                             )                                                                 
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 40, 40, 256)  0          ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2d_2687 (Conv2D)           (None, 158, 158, 80  5120        ['max_pooling2d_199[0][0]']      
          

...

                                                                                                  
 conv5_block1_2_conv (Conv2D)   (None, 20, 20, 512)  2359296     ['conv5_block1_2_pad[0][0]']     
                                                                                                  
 conv2d_2754 (Conv2D)           (None, 38, 38, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_2758 (Conv2D)           (None, 38, 38, 192)  258048      ['activation_2705[0][0]']        
                                                                                                  
 block_14_expand (Conv2D)       (None, 8, 8, 960)    153600      ['block_13_project_BN[0][0]']    
                                                                                                  
 conv5_block1_2_bn (BatchNormal  (None, 20, 20, 512)  2048       ['conv5_block1_2_conv[0][0]']    
 ization) 

 conv2d_2764 (Conv2D)           (None, 18, 18, 448)  573440      ['mixed8[0][0]']                 
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 8, 8, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 20, 20, 512)  2048       ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 batch_normalization_2712 (Batc  (None, 18, 18, 448)  1344       ['conv2d_2764[0][0]']            
 hNormalization)                                                                                  
          

 hNormalization)                                                                                  
                                                                                                  
 batch_normalization_2715 (Batc  (None, 18, 18, 384)  1152       ['conv2d_2767[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 conv2d_2768 (Conv2D)           (None, 18, 18, 192)  245760      ['average_pooling2d_271[0][0]']  
                                                                                                  
 block_15_depthwise_relu (ReLU)  (None, 8, 8, 960)   0           ['block_15_depthwise_BN[0][0]']  
                                                                                                  
 conv5_block2_out (Add)         (None, 20, 20, 2048  0           ['conv5_block1_out[0][0]',       
          

 block_16_expand_relu (ReLU)    (None, 8, 8, 960)    0           ['block_16_expand_BN[0][0]']     
                                                                                                  
 conv5_block3_2_pad (ZeroPaddin  (None, 22, 22, 512)  0          ['conv5_block3_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv2d_2770 (Conv2D)           (None, 18, 18, 384)  786432      ['mixed9[0][0]']                 
                                                                                                  
 conv2d_2774 (Conv2D)           (None, 18, 18, 384)  1548288     ['activation_2721[0][0]']        
                                                                                                  
 block_16_depthwise (DepthwiseC  (None, 8, 8, 960)   8640        ['block_16_expand_relu[0][0]']   
 onv2D)   

                                                                                                  
 activation_2723 (Activation)   (None, 18, 18, 384)  0           ['batch_normalization_2723[0][0]'
                                                                 ]                                
                                                                                                  
 activation_2724 (Activation)   (None, 18, 18, 384)  0           ['batch_normalization_2724[0][0]'
                                                                 ]                                
                                                                                                  
 batch_normalization_2725 (Batc  (None, 18, 18, 192)  576        ['conv2d_2777[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 Conv_1 (C